## Graphs and tables for the weekly fundamentals review

In [3]:
import Quandl
import numpy as np
import pandas as pd
import datetime as dt  
from dateutil.relativedelta import *
import calendar
import requests
from pprint import pprint
import json

In [4]:
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.io import output_notebook, show, vform
output_notebook()

Loading BokehJS ...

In [5]:
pd.options.display.float_format = '{:.3f}'.format
float_formatter = lambda x: "%.3f" % x
#np.set_printoptions(formatter={'float_kind':float_formatter})

### Notes

* [relative delta](http://dateutil.readthedocs.org/en/latest/index.html) in the dateutil package is awesome
* [COT](https://www.quandl.com/data/CFTC?keyword=) is updated on tuesdays
* [EIA](http://www.eia.gov/opendata/) API documentation and links
* [Requests](http://docs.python-requests.org/en/master/) HTTP for Python
* [EIA data with python](https://quantcorner.wordpress.com/2014/11/18/downloading-eias-data-with-python/)
* [tweepy](http://docs.tweepy.org/en/v3.5.0/index.html) documentation
    * [twitter example 1](http://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/word_cloud/Mining_Twitter_Example.ipynb)
    * [twitter example 2](https://gist.github.com/arulrajnet/0b71842b573d81c7bc23)
    * also search github git for examples and saved a notbook in my python folder

### Improvements

* store the commodityMonths dict somewhere I can just load it for regular use
* should pull more data with NG prices query and make a chart or graph form it. The NG query is set up to do convergence, but that requires ICE data and I cant automate with that so make something to replace.
* be able to do multiple weeks for 'this week in H' table

### Set Up variables 

In [6]:
#from datetime get today's date
today = dt.date.today()

#today string for display
today_str = today.strftime("%m/%d/%Y")

#sixty days ago for COT data
sixty = today + relativedelta(months=-2)

#six years ago for benchmarking EIA data
sixyears = today + relativedelta(years=-6)

In [7]:
#last fri for NG price call from quandl
lastFriday = today + relativedelta(weekday=FR(-1))

#last mon for NG price call from quandl
lastMonday = today + relativedelta(weekday=MO(-2))

#picking months from last wed instead of mon prevents picking weeks with zero prompt month prices
#this could be more thought out, or it'd be nice to have a multi year bidweek calendar
lastWednesday = today + relativedelta(weekday=WE(-1))

In [8]:
#dict of commodity months
commodityMonths = {'January':'F','February':'G','March':'H','April':'J','May':'K','June':'M',
'July':'N','August':'Q','September':'U','October':'V','November':'X','  December':'Z'}

In [9]:
#prompt month is the next month, str version used for labeling columns
prompt = lastWednesday + relativedelta(months=+1)
promptStr = calendar.month_name[prompt.month]

#prompt +1 second delivery month
second = lastWednesday + relativedelta(months=+2)
secondStr = calendar.month_name[second.month]

#prompt +3 third delivery month
third = lastWednesday + relativedelta(months=+3)
thirdStr = calendar.month_name[third.month]

In [10]:
#used to authomate symbols in quandl query
promptYear = prompt.year
secondYear = second.year
thirdYear = third.year

In [11]:
#used to automate symbols in quandl query
promptC = commodityMonths[promptStr]
secondC = commodityMonths[secondStr]
thirdC = commodityMonths[thirdStr]

In [12]:
# vars for getting price data
#begin and end date of data
bw = lastMonday.strftime("%m/%d/%Y")
ew = lastFriday.strftime("%m/%d/%Y")

#symbols change month letter and year when necessary
first = "CME/NG"+promptC+str(promptYear)+".6"
second = "CME/NG"+secondC+str(secondYear)+".6"
third = "CME/NG"+thirdC+str(thirdYear)+".6"

#rename change month name when necessary
firstn = "NG " + promptStr
secondn = "NG" + secondStr
thirdn = "NG" + thirdStr

#vars for getting OI data
sixty = sixty.strftime("%m/%d/%Y")

### Get data from quandl

In [13]:
#the .6 after the contract name gives specific column
NG = Quandl.get([first,second,third], authtoken="UDaCz3fqxxJUAL3-WWmD", trim_start=bw, trim_end=ew, returns="pandas")

In [14]:
#leave out trim end defaults to most recent
COT = Quandl.get("CFTC/H_FO_ALL", authtoken="UDaCz3fqxxJUAL3-WWmD",trim_start=sixty, returns="pandas")

### H this week

In [15]:
#cull the columns
NG.columns = [firstn,secondn,thirdn]
OI = COT[['Open Interest']]

In [16]:
#calc max price last week
HMax = float_formatter(NG[firstn].max())

#return date of start of week
Start = NG.index[0].strftime("%m/%d/%Y")

In [17]:
#return price of first day of week, but not if its nan
NGFirst = NG[np.isfinite(NG[firstn])]
NGFirst = NGFirst[[firstn]]

In [18]:
#calc percent change from first to last, also using shortened NGFirst from above
NGchng = NGFirst[firstn].pct_change(periods=len(NGFirst.index)-1)

In [19]:
#return OI of last entry in COT data (most recent date)
OIlast = OI.ix[-1]

In [20]:
#calc percent change in OI from this week to last week
OIchng = OI['Open Interest'].pct_change(periods=1)

In [21]:
#reformat everything and define dict for display
Week1 = dict(
    Start = [Start],
    Contract = [firstn],
    Open = [float_formatter(NGFirst.ix[0])],
    High = [float_formatter(NGFirst.max())],
    Low = [float_formatter(NGFirst.min())],
    Close = [float_formatter(NGFirst.ix[-1])],
    changePrice = ["{:.2%}".format(NGchng.ix[len(NGchng.index)-1])],
    changeOI  = ["{:.2%}".format(OIchng.ix[len(OIchng.index)-1])],
    OI  = ["{:,}".format(OIlast[0].astype(int))],
    )
    

In [22]:
#bokeh to output nice looking table
source = ColumnDataSource(Week1)

columns = [
        TableColumn(field="Start", title="Start"),
        TableColumn(field="Contract", title="Contract"),
        TableColumn(field="Open", title="Open"),
        TableColumn(field="High", title="High"),
        TableColumn(field="Low", title="Low"),
        TableColumn(field="Close", title="Close"),
        TableColumn(field="changePrice", title="%change Price"),
        TableColumn(field="OI", title="OI"),
        TableColumn(field="changeOI", title="%change OI"),
    ]

data_table = DataTable(source=source, columns=columns, width=850, height=75)

show(vform(data_table))

### Storage Report

In [68]:
#api url for series name Weekly Lower 48 States Natural Gas Working Underground Storage, Weekly
storageURL = 'http://api.eia.gov/series/?api_key=B617CC071819064DEFFF9ED570FDB75A&series_id=NG.NW2_EPG0_SWO_R48_BCF.W&out=JSON'

In [69]:
#get requests data from specificied url
storage = requests.get(storageURL)

In [70]:
#the request was for json formatting so uses requests' JSON decoder
storage_json = storage.json()

#storage_json is a nested dict, so take a look at it and find my way into the sub-dict with the data
date_series = storage_json['series'][0]['data']

In [71]:
#loop through that date_series dict and make and array out of the date entries
endi = len(date_series)
date = []

for i in range(endi):
    date.append(date_series[i][0])

In [72]:
#convert that bad boy to a data frame
df = pd.DataFrame(data=date)
df.columns = ['Date']

In [73]:
#loop through the rest of the dict with and put the data in
lenj = len(storage_json)-1

for j in range (lenj):
    data_series = storage_json['series'][0]['data']
    data = []
    endk = len(date_series) 
    for k in range (endk):
        data.append(data_series[k][1])
    df[j] = data

In [74]:
#rename columns
df.columns = ['date','storageUS']

#substring date to use for making a python datevariable, the infer datetime thing didnt work 
df['year'] = df['date'].apply(lambda t: t[0:4])
df['year'] = df['year'].astype(int)

df['month'] = df['date'].apply(lambda t: t[4:6])
df['month'] = df['month'].astype(int)

df['day'] = df['date'].apply(lambda t: t[-2:])
df['day'] = df['day'].astype(int)

In [75]:
#date formatted as datetime
df['date_f'] = pd.to_datetime(df.year*10000 + df.month*100 + df.day, format='%Y%m%d')

In [76]:
#week of year to make y-y comparisons
df['week'] = df['date_f'].dt.week

In [77]:
#move the index to date time
df.set_index(['date_f'], inplace = True)
#sort by datetime acending, bc the data comes in most rescent first
df = df.sort_index()
#calc weekly storage change
df['lastWeek'] = df['storageUS'].shift()
df['change'] = df['storageUS'] - df['lastWeek']

In [78]:
#make it so the index label doesnt appear, just looks prettier
df.index.name = None

In [79]:
#subset the data so for most recent six years
#six because my code to calc 5 year average takes in current week, will use shift and rolling avg to get 5year
df = df[pd.to_datetime(df.year*10000 + df.month*100 + df.day, format='%Y%m%d')>= sixyears]

In [93]:
#rolling average storage volume grouped by week
df['avgChange'] = df.groupby('week')['change'].apply(pd.rolling_mean,len(df),min_periods=1)
df['avgStorageUS'] = df.groupby('week')['storageUS'].apply(pd.rolling_mean,len(df),min_periods=1)


In [124]:
# calc non inclusive five year, sort by week then by date then create var based on shifted avgstorage us
#sort by index back to normal
df = df.sort_values(['week','date'],ascending=[1,1])
df['storage_5yr'] = df['avgStorageUS'].shift()
df = df.sort_index()

In [131]:
#cal percent diff from 5 year average
df['percentDiff'] = (df['storageUS'] - df['storage_5yr'])/df['storage_5yr']

In [132]:
recent = df.tail()
recent

,date,storageUS,year,month,day,week,lastWeek,change,avgChange,avgStorageUS,storage_5yr,percentDiff
2016-01-29,20160129,2934,2016,1,29,4,3086.000,-152.000,-166.667,2630.667,2570.000,0.142
2016-02-05,20160205,2864,2016,2,5,5,2934.000,-70.000,-142.000,2488.667,2413.600,0.187
2016-02-12,20160212,2701,2016,2,12,6,2864.000,-163.000,-178.833,2309.833,2231.600,0.210
2016-02-19,20160219,2584,2016,2,19,7,2701.000,-117.000,-141.167,2168.667,2085.600,0.239
2016-02-26,20160226,2536,2016,2,26,8,2584.000,-48.000,-117.167,2051.500,1954.600,0.297


In [130]:
x = df[df['week']==5]
x

,date,storageUS,year,month,day,week,lastWeek,change,avgChange,avgStorageUS,storage_5yr,percentDiff
2011-02-04,20110204,2144,2011,2,4,5,2355.000,-211.000,-211.000,2144.000,2630.667,-0.227
2012-02-03,20120203,2888,2012,2,3,5,2966.000,-78.000,-144.500,2516.000,2144.000,0.296
2013-02-01,20130201,2684,2013,2,1,5,2802.000,-118.000,-135.667,2572.000,2516.000,0.065
2014-01-31,20140131,1924,2014,1,31,5,2184.000,-260.000,-166.750,2410.000,2572.000,-0.269
2015-01-30,20150130,2428,2015,1,30,5,2543.000,-115.000,-156.400,2413.600,2410.000,0.007
2016-02-05,20160205,2864,2016,2,5,5,2934.000,-70.000,-142.000,2488.667,2413.600,0.181


## Twitter

__some lists for starters__

_nat gas_
* [Peter Gardett](https://twitter.com/petergardett/lists)
* [EIA](https://twitter.com/EIAgov/lists)
* [InterfaxEnergy](https://twitter.com/InterfaxEnergy)
* [Cohan](https://twitter.com/cohan_ds/lists)

_power_
* https://twitter.com/innotap
* https://twitter.com/Brad_E_Foster
* https://twitter.com/rtoinsider
* https://twitter.com/MohitShrestha
* https://twitter.com/nrgytariffxpert


In [192]:
import tweepy

pd.set_option("display.max_rows",1000)
pd.set_option("display.max_columns",50)
pd.set_option("display.max_colwidth",150)

In [135]:
consumer_key = 'UvB8nZGq2m4GcNk64Pv3U9kln'
consumer_secret = 'FhX4W6SV1dGT0GZSBvTzHzt2eoYYS27SrqqKm4CGR3eiqwQURc'

access_token ='18363853-642KR2i2Mq9ZB5aWwZbpziMUX39IfTTdsrlawUXKG'
access_token_secret = 'dwGaU0xEMRJta5Qoos2k47tTEPYR3HwGp0D295nYV6ZxS'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [138]:
public_tweets = api.home_timeline()
for (idx, tweet) in enumerate(public_tweets[0:3]): #First 3 tweets in my public feed
    print ('TWEET %s:\n\n\t%s\n\n' % (idx, tweet.text))

TWEET 0:

	Why the polls underrated Bernie Sanders in Michigan https://t.co/PjMAqcad8T


TWEET 1:

	Barack Obama explained what’s wrong with Donald Trump in a speech in 2005! Watch: https://t.co/2y9R34HY2Q https://t.co/1qJRV2Cnzf


TWEET 2:

	#Ecuador exports roughly 70% of the #crude #oil it produces https://t.co/jHugBvDqdF #InternationalEnergy https://t.co/GnkTMjQbs0




In [187]:
pprint(tweet.__dict__.keys())

dict_keys(['id_str', '_api', '_json', 'entities', 'contributors', 'in_reply_to_status_id', 'is_quote_status', 'place', 'geo', 'lang', 'source_url', 'user', 'id', 'favorited', 'in_reply_to_status_id_str', 'coordinates', 'retweeted', 'in_reply_to_user_id', 'favorite_count', 'text', 'retweet_count', 'truncated', 'metadata', 'created_at', 'in_reply_to_screen_name', 'source', 'in_reply_to_user_id_str', 'author'])


In [147]:
top10 = api.trends_place(id=2379574)
top10[0]['trends']

[{'name': '#Texans',
  'promoted_content': None,
  'query': '%23Texans',
  'tweet_volume': 17539,
  'url': 'http://twitter.com/search?q=%23Texans'},
 {'name': 'Broncos',
  'promoted_content': None,
  'query': 'Broncos',
  'tweet_volume': 76096,
  'url': 'http://twitter.com/search?q=Broncos'},
 {'name': 'Jets',
  'promoted_content': None,
  'query': 'Jets',
  'tweet_volume': 57052,
  'url': 'http://twitter.com/search?q=Jets'},
 {'name': 'Patrick Kane',
  'promoted_content': None,
  'query': '%22Patrick+Kane%22',
  'tweet_volume': None,
  'url': 'http://twitter.com/search?q=%22Patrick+Kane%22'},
 {'name': '#GetElectedIn3Words',
  'promoted_content': None,
  'query': '%23GetElectedIn3Words',
  'tweet_volume': 12498,
  'url': 'http://twitter.com/search?q=%23GetElectedIn3Words'},
 {'name': '#TheTriggering',
  'promoted_content': None,
  'query': '%23TheTriggering',
  'tweet_volume': 155259,
  'url': 'http://twitter.com/search?q=%23TheTriggering'},
 {'name': '#SonyIsDroppingDrLukeParty',
  '

In [149]:
for trend in top10[0]['trends']:
    print (trend['name'], trend['url'])

#Texans http://twitter.com/search?q=%23Texans
Broncos http://twitter.com/search?q=Broncos
Jets http://twitter.com/search?q=Jets
Patrick Kane http://twitter.com/search?q=%22Patrick+Kane%22
#GetElectedIn3Words http://twitter.com/search?q=%23GetElectedIn3Words
#TheTriggering http://twitter.com/search?q=%23TheTriggering
#SonyIsDroppingDrLukeParty http://twitter.com/search?q=%23SonyIsDroppingDrLukeParty
Ayesha Curry http://twitter.com/search?q=%22Ayesha+Curry%22
#WednesdayWisdom http://twitter.com/search?q=%23WednesdayWisdom
Brandon Ingram http://twitter.com/search?q=%22Brandon+Ingram%22
Kaepernick http://twitter.com/search?q=Kaepernick
Cooney http://twitter.com/search?q=Cooney
Elway http://twitter.com/search?q=Elway
Carly Fiorina http://twitter.com/search?q=%22Carly+Fiorina%22
Di Maria http://twitter.com/search?q=%22Di+Maria%22
Sanu http://twitter.com/search?q=Sanu
Cat Barber http://twitter.com/search?q=%22Cat+Barber%22
Damon Harrison http://twitter.com/search?q=%22Damon+Harrison%22
Jon Vo

In [208]:
results = api.search(q='Henry Hub', count=5, lang='en')
print (type(results))

<class 'tweepy.models.SearchResults'>


In [186]:
for tweet in results:
    print (tweet.created_at, '\n',  tweet.text, '\n\n')

2016-03-09 21:33:29 
 @cohan_ds @N565UA @CycloneCharlie8 If you followed our charts u know our tgt you said henry Hub was at a price it aint been in 20 years 


2016-03-09 21:00:39 
 @N565UA @CycloneCharlie8 @TradersCom
EIA now forecasts Henry Hub rising &gt;40% in 2016 from current $1.56.
https://t.co/fHOaNubeHm 


2016-03-09 20:47:20 
 Henry Hub #natgas prices, now $1.56, not following #EIAErrors.
https://t.co/q7iWy9GW93
@CycloneCharlie8 @TradersCom https://t.co/q2BdWatEFM 


2016-03-09 20:40:04 
 EIA forecasts 2016 avg #natgas $2.25/MMBtu https://t.co/uWLp3o9nTp
Henry Hub price now $1.56
https://t.co/Pm0ZWhDKCU https://t.co/uV2nh1TM1x 


2016-03-09 18:45:42 
 @concur #ConcurFusion Three great sessions yesterday. Shout out to Jena Henry at the SMB hub for the great help. Productive convention 




In [193]:
tweets = pd.DataFrame()

tweets["text"] = [tweet.text for tweet in results]

tweets

,text
0,@cohan_ds @N565UA @CycloneCharlie8 If you followed our charts u know our tgt you said henry Hub was at a price it aint been in 20 years
1,@N565UA @CycloneCharlie8 @TradersCom\nEIA now forecasts Henry Hub rising &gt;40% in 2016 from current $1.56.\nhttps://t.co/fHOaNubeHm
2,"Henry Hub #natgas prices, now $1.56, not following #EIAErrors.\nhttps://t.co/q7iWy9GW93\n@CycloneCharlie8 @TradersCom https://t.co/q2BdWatEFM"
3,EIA forecasts 2016 avg #natgas $2.25/MMBtu https://t.co/uWLp3o9nTp\nHenry Hub price now $1.56\nhttps://t.co/Pm0ZWhDKCU https://t.co/uV2nh1TM1x
4,@concur #ConcurFusion Three great sessions yesterday. Shout out to Jena Henry at the SMB hub for the great help. Productive convention


In [157]:
for tweet in api.search(q='Henry Hub', count=5, show_user=False, lang='en'):
    print (tweet.created_at, '\n',  tweet.text, '\n\n')

2016-03-09 21:00:39 
 @N565UA @CycloneCharlie8 @TradersCom
EIA now forecasts Henry Hub rising &gt;40% in 2016 from current $1.56.
https://t.co/fHOaNubeHm 


2016-03-09 20:47:20 
 Henry Hub #natgas prices, now $1.56, not following #EIAErrors.
https://t.co/q7iWy9GW93
@CycloneCharlie8 @TradersCom https://t.co/q2BdWatEFM 


2016-03-09 20:40:04 
 EIA forecasts 2016 avg #natgas $2.25/MMBtu https://t.co/uWLp3o9nTp
Henry Hub price now $1.56
https://t.co/Pm0ZWhDKCU https://t.co/uV2nh1TM1x 


2016-03-09 18:45:42 
 @concur #ConcurFusion Three great sessions yesterday. Shout out to Jena Henry at the SMB hub for the great help. Productive convention 


2016-03-09 17:50:25 
 @ConcurFusion. Special thanks to SMB hub and Jena Henry for the help yesterday. Made the trip worth it. #aboveandbeyond 




In [166]:
results[0].user

User(location='Houston, TX', friends_count=2615, id_str='2505359724', _api=<tweepy.api.API object at 0x0000000008EDCEF0>, has_extended_profile=True, _json={'location': 'Houston, TX', 'friends_count': 2615, 'profile_use_background_image': False, 'id_str': '2505359724', 'description': 'Air Climate Energy education and research via https://t.co/wl061pTe3D. NASA AQAST. #ScientistReporter for The Hill et al. #SurplusSolar. Dad of 2 awesome kids.', 'profile_sidebar_fill_color': '000000', 'created_at': 'Sun May 18 19:51:11 +0000 2014', 'has_extended_profile': True, 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/2505359724/1455292491', 'contributors_enabled': False, 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_color': '000000', 'default_profile_image': False, 'name': 'Daniel Cohan', 'default_profile': False, 'lang': 'en', 'favourites_count': 3708, 'profile_sidebar_border_color': '000000', 'following': False, 'verif

In [206]:
new_tweets = api.user_timeline(screen_name='GenscapeWx', count=5)

In [223]:
for tweet in new_tweets:
    print (tweet.text, ' ', tweet.source_url)

With models offering a wide range of scenarios, will we have El Nino, La Nina, or neutral conditions this summer?   http://twitter.com
More #snow for #California and greater West nxt 10 days. Good for #drought, #hydropower.
@genscape #renewables #cawx https://t.co/JY1SJhxq0N   http://twitter.com
The difference a week makes! Avg #snow depth up 11", nearly 2x areal coverage in #SierraNevada mtns.
@genscape #cawx https://t.co/11RwTOsXDC   http://twitter.com
More wild #txwx on the way through early nxt wk; #flood threat into Fri followed by anomalous #heat Mon. @genscape https://t.co/823wYjN4hI   http://twitter.com
Pronounced East Coast ridge drives #record #warmth &amp; weak #energy demand into Mid-March. @genscape https://t.co/0X9QWF4rRj   http://twitter.com


In [216]:
print(new_tweets[0].__dict__.keys())

dict_keys(['id_str', '_api', '_json', 'entities', 'contributors', 'in_reply_to_status_id', 'is_quote_status', 'place', 'geo', 'lang', 'source_url', 'user', 'id', 'favorited', 'in_reply_to_status_id_str', 'coordinates', 'retweeted', 'in_reply_to_user_id', 'favorite_count', 'text', 'retweet_count', 'truncated', 'created_at', 'in_reply_to_screen_name', 'source', 'in_reply_to_user_id_str', 'author'])


### Boneyard

In [ ]:
# C = df.groupby(['week'])['change'].mean()
# T = df.groupby(['week'])['storageUS'].mean()

# #stats = pd.DataFrame(C,T)
# #stats.index.name = 'week'

#calculations done outside the dict
# HLast = float_formatter(NGFirst.ix[-1])
# HFirst = float_formatter(NGFirst.ix[0])
# float_formatter(NGFirst.max())
# float_formatter(NGFirst.min())
# NGchng.ix[len(NGchng.index)-1]
# OIchng.ix[len(OIchng.index)-1]


#"{:,}".format(x[0].astype(int))
#NGMar = NGMar.resample('5D',how='ohlc')